# Level 4: Agentic & MCP (Medium Difficulty)

This tutorial is aimed at those already familiar with basic Agentic workflows. It is meant to showcase  **sequential tool calls** or **conditional logic** within the context of an agentic workflow.

## Overview

In this tutorial we will be connecting to a llama-stack instance, building an agent with various tools available to it, and inferencing against the agent.

## Prerequisites

Before starting, ensure you have the following:
- Access to an openshift cluster with A deployment of the [openshift MCP server](https://github.com/opendatahub-io/llama-stack-on-ocp/tree/main/mcp-servers/openshift) (see the [deployment manifests](https://github.com/opendatahub-io/llama-stack-on-ocp/tree/main/kubernetes/mcp-servers/openshift-mcp) for assistance with this).
- User variables configured (see section `Setting your ENV variables` below).
- A Tavily API key is required. You can register for one at https://tavily.com/home.

## General Setup - Agnostic to all Queries

These steps will be the same for all 3 queries, with the exception of inputing the Tavily API key - this will only be used for query 2, and so could be ommitted if one only wants to run demos 1 or 3.

### Setting your ENV variables:

As mentioned above, for this demo there are a few ENV variables that need to set:
- `REMOTE` (boolean): dictates if you are using a remote llama-stack instance.
- `REMOTE_BASE_URL` (string): the URL for your llama-stack instance if using remote connection.
- `TAVILY_SEARCH_API_KEY` (string): your API key for tavily search. One can get one by going to: https://tavily.com/home.
- `REMOTE_OCP_MCP_URL` (string): the URL for your Openshift MCP server. If the client does not find the tool registered to the llama-stack instance, it will use this URL to register the Openshift tool (used in demos 1 and 3).
- `USE_PROMPT_CHAINING` (boolean): dictates if the prompt should be formatted as 3 separate prompts to isolate the steps. This is an option because a single inference call is able to do the search and draft the email, but by destructuring this into sub-steps where the LLM can focus on a single task at a time can theoretically increase performance. Additionally while this task can be accomplished from a single prompt, other more complex questions may not be doable in this way. This prompt-chaining example is meant to demonstrate how one might do this.

### Installing dependencies

This code requires `llama-stack` and the `llama-stack-client`, both at version `0.1.9`. Lets begin by installing them:

In [ ]:
!pip install llama-stack-client==0.1.9 llama-stack==0.1.9

### Configuring logging

Now that we have our dependencies, lets setup logging for the application:

In [ ]:
from llama_stack_client.lib.agents.event_logger import EventLogger
import logging

logger = logging.getLogger(__name__)
if not logger.hasHandlers():  
    logger.setLevel(logging.INFO)
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(message)s')
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

### Connecting to llama-stack server

For the llama-stack instance, you can either run it locally or connect to a remote llama-stack instance.

#### Remote llama-stack

- For remote, be sure to set `remote` to `True` and populate the `remote_llama_stack_endpoint` variable with your llama-stack remote.
- [Remote Setup Guide](https://github.com/opendatahub-io/llama-stack-on-ocp/tree/main/kubernetes)

#### Local llama-stack
- For local, be sure to set `remote` to `False` and validate the `local_llama_stack_endpoint` variable. It is based off of the default llama-stack port which is `8321` but is configurable with your deployment of llama-stack.
- [Local Setup Guide](https://github.com/redhat-et/agent-frameworks/tree/main/prototype/frameworks/llamastack)

In [ ]:

import os

remote = os.getenv("REMOTE") # Use the `remote` variable to switching between a local development environment and a remote kubernetes cluster.
model="meta-llama/Llama-3.2-3B-Instruct"

if remote and remote is True:
    base_url = os.getenv("REMOTE_BASE_URL")
else:
    base_url = "http://localhost:8321"

tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY") # Replace with your Tavily API key (required for demo 2)

from llama_stack_client import LlamaStackClient

client = LlamaStackClient(
    base_url=base_url,
    provider_data={
        "tavily_search_api_key": tavily_search_api_key # This is required for demo 2
    })
logger.info(f"Connected to Llama Stack server @ {base_url} \n")

### Validate tools are available in our llama-stack instance

When an instance of llama-stack is redeployed your tools need to re-registered. Also if a tool is already registered with a llama-stack instance, if you try to register one with the same `toolgroup_id`, llama-stack will throw you an error.

For this reason it is recommended to include some code to validate your tools and toolgroups. This is where the `mcp_url` comes into play. The following code will check that both the `builtin::websearch` and the `mcp::openshift` tools are registered as tools, but if the `mcp::openshift` tool is not listed there, it will attempt to register it using the mcp url.

If you are running the MCP server from source, the default value for this is: `http://localhost:8000/sse`.

If you are running the MCP server from a container, the default value for this is: `http://host.containers.internal:8000/sse`.

In [ ]:
mcp_url = os.getenv("REMOTE_OCP_MCP_URL") # Optional: enter your MCP server url here

registered_tools = client.tools.list()
registered_toolgroups = [t.toolgroup_id for t in registered_tools]
if  "builtin::websearch" not in registered_toolgroups: # Required for demo 2
    client.toolgroups.register(
        toolgroup_id="builtin::websearch",
        provider_id="tavily-search",
        args={"max_results": 10},
    )
    
if "mcp::openshift" not in registered_toolgroups: # required for demos 1 and 3
    client.toolgroups.register(
        toolgroup_id="mcp::openshift",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":mcp_url},
    )

## Query 1: (Agentic) `Check the status of my OpenShift cluster. If it’s running, create a new pod named test-pod in the dev namespace.`

In [ ]:
print("some_python_code_for_l4_q1")

## Query 2: (Agentic): `Search for the latest Red Hat OpenShift version on the Red Hat website. Summarize the version number and draft a short email to my team.`

Previously, we instantiated our llama-stack client with our tavily search API key, and connected to our instance of that llama-stack client, before ensuring our required tools and toolgroups are registered to that llamastack instance.

Now we can create our agent, and start our agent sessions. As described above, we will need to decide if we want to use prompt-chaining or not. While its not required for this medium complexity query, as the model is good enough to do both the tool call and inference in one step, this may not be the case for other more complex queries.

In [ ]:
from llama_stack_client.lib.agents.agent import Agent

agent = Agent(
    client=client,
    model=model,
    instructions="""You are a helpful AI assistant, responsible for helping me find and communicate information back to my team.
    You have access to a number of tools.
    Whenever a tool is called, be sure return the Response in a friendly and helpful tone.
    When you are asked to search the web you must use a tool.
    When signing off on emails, please be sure to include: - Sent from my llama-stack agent in the signature
    """,
    tools=["builtin::websearch", "mcp::openshift"],
    tool_config={"tool_choice":"auto"},
    sampling_params={
        "max_tokens":4096,
        "strategy": {"type": "greedy"},
    }
)

session_id = agent.create_session(session_name="Draft_email_with_latest_OCP_version")

prompt_chaining = os.getenv("USE_PROMPT_CHAINING") # Decide if prompt should be destructured into multiple turns or 

if prompt_chaining and prompt_chaining is True:
    prompts = [
        """Search for the web for the latest Red Hat OpenShift version on the Red Hat website.""",
        """Summarize the latest Red Hat OpenShift version number and any significant features, fixes, or changes that occure in this version.""",
        """Draft and format an email to convey this information to my team members."""
    ]
    for i, prompt in enumerate(prompts):    
        turn_response = agent.create_turn(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            session_id=session_id,
            stream=True,
        )
        logger.info(f"========= Turn: {i} =========")
        for log in EventLogger().log(turn_response):
            log.print()
else:
    prompt = """Search for the web for the latest Red Hat OpenShift version on the Red Hat website. Summarize the version number and draft an email to convey this information."""
    turn_response = agent.create_turn(
        messages=[
            {
                "role":"user",
                "content": prompt
            }
        ],
        session_id=session_id,
        stream=True,
    )
    for log in EventLogger().log(turn_response):
        log.print()

inference> 
,tool_execution> Tool:brave_search Args:{'query': 'latest Red Hat OpenShift version'}
,tool_execution> Tool:brave_search Response:{"query": "latest Red Hat OpenShift version", "top_k": [{"title": "Red Hat Enhances Security and Virtualization Experience with Latest ...", "url": "https://www.redhat.com/en/about/press-releases/red-hat-enhances-security-and-virtualization-experience-latest-version-red-hat-openshift", "content": "Red Hat, Inc., the world's leading provider of open source solutions, today announced the general availability of Red Hat OpenShift 4.18, the latest version of the industry\u2019s leading hybrid cloud application platform powered by Kubernetes. Red Hat OpenShift 4.18 introduces new features and capabilities designed to streamline operations and security across IT environments and deliver greater consistency to all applications, from cloud-native and AI-enabled to virtualized and traditional. Additionally, for users looking for virtualization in the publ

#### Output Analysis

Lets step through the output to further understands whats happening in this Agentic demo.

1. First the LLM sends off a tool call to the `brave_search` to lookup the latest version of Red Hat Openshift.
2. The LLM recieves the response from the tool call, results of the search, along with the orrigional query. Each search result has:
    - Title of the webpage
    - Website URL
    - Relevant content exerpt from the search result
    - A score, quanitifying how relevant is the search result to the search query
    - raw_content from the page
3. Finally, this context gets passed back to the LLM for the final inference. The inference result starts my responding to my initial question with some background, and then finally drafting the email that was requested. This example was ran without prompt-chaining.

## Query 3: (MCP) `Review OpenShift logs for pods pod-123 and pod-456. Categorize each as ‘Normal’ or ‘Error’. If any show ‘Error’, send a Slack message to the ops team. Otherwise, show a simple summary.`

In [ ]:
print("some_python_code_for_l4_q3")